# Title

In [1]:
#imports
import acquire as a
import prepare as p
import explore as e
import model_functions as m 


# Acquire

In [2]:
df = a.get_heart()

# Prepare

In [3]:
df = p.rename_cols(df)
df = p.replace_cat_values(df)
df = p.split_data(df, 'high_risk_of_mi')

# Explore

# Exploration Summary

# Modeling

In [5]:
df = m.df_classification_ready(df, 'sex_female')


TypeError: unhashable type: 'DataFrame'

In [ ]:

train, val, test = p.split_data(df, 'high_risk_of_mi')

X_Train, y_Train, X_val, y_val, X_test, y_test = m.isolate_target(train, val, test, 'high_risk_of_mi')

#### Best 3 Models

#### Test Model

# Modeling Wrap

# Conclusion

#### Summary

#### Recommendations

#### Next Steps